In [2]:
import pydantic

In [3]:
# Type validation without using pydantic
class User:
    def __init__(self,id: int, name='Jane Doe'):
        if not isinstance(id,int):
            raise TypeError(f'Expected id to be an int, got {type(id).__name___}')
        if not isinstance(name,str):
            raise TypeError(f'Expected name to be a str, got {type(name).__name__}')
        
        self.id = id
        self.name = name
try:
    user = User(id='123')
except TypeError:
    print("Type Error")

AttributeError: type object 'str' has no attribute '__name___'

In [4]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

In [8]:
user = User(id='123') #parses string to integer 
print(user.id)

123


In [11]:
print(user.model_fields_set)
print(user.model_dump_json()) #prints json notation
print(user.model_dump()) # prints it as a dictionary notation

{'id'}
{"id":123,"name":"Jane Doe"}
{'id': 123, 'name': 'Jane Doe'}


# Nested Model

In [12]:
from typing import List, Optional
from pydantic import BaseModel

class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None

class Restaurant(BaseModel):
    name: str
    location: str
    foods: List[Food]

restaurant_instance = Restaurant(
    name="Tasty bites",
    location="California",
    foods=[
        {"name":"Cheese Pizza", "price":12.20,"ingredients":["Cheese","Tomato Sauce","Pepperoni"]}
    ]
)

print(restaurant_instance.model_dump())
print(restaurant_instance.model_dump_json())


{'name': 'Tasty bites', 'location': 'California', 'foods': [{'name': 'Cheese Pizza', 'price': 12.2, 'ingredients': ['Cheese', 'Tomato Sauce', 'Pepperoni']}]}
{"name":"Tasty bites","location":"California","foods":[{"name":"Cheese Pizza","price":12.2,"ingredients":["Cheese","Tomato Sauce","Pepperoni"]}]}


In [14]:
!pip install pydantic[email]

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [email-validator]1/2 [email-validator]


In [ ]:
from typing import List
from pydantic import BaseModel, EmailStr,PositiveInt,conlist,Field,HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr

class Owner(BaseModel):
    name: str
    position: str
    email: EmailStr

class Restaurant(BaseModel):
    name: str = Field(...) #indicate the required field
    owner: Owner
    address: Address
    employees: conlist(Employee, min_length=2) #list of employees with min length 2
    number_of_seats: PositiveInt
    deliver: bool
    website: HttpUrl #check if url is http




In [25]:
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name": "John Doe",
        "email": "hello@example.com",
        "position":"owner"
    },
    address = {
        "street":"123 Baker Street",
        "city": "Tasty town",
        "state": "TS",
        "zip_code": "12345",
    },
    employees = [
        {
            "name":"Jane Doe",
            "position": "Head Chef",
            "email": "jane@example.com",
        },
        {
            "name": "Mike Cary",
            "position": "Asst. Chef",
            "email": "mike@example.com"
        }
    ],
    number_of_seats=50,
    deliver=True,
    website="https://google.com"
)

# Feild Validators

In [ ]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator('name')
    @classmethod

    def name_must_contain_space(cls, v:str) -> str:
        if ' ' not in v:
            raise ValueError("Onwner name must contain space")
        return v.upper()
    
try:
    owner_instance = Owner(name="John Doe", email="hello@example.com")
except ValueError as e:
    print(e)
print(owner_instance)

name='JOHN DOE' email='hello@example.com'
